In [1]:
# !pip install catboost

In [2]:
import pandas as pd
import sys
# from google.colab import files
# import io 
import re
import datetime
import ast
import json

from sklearn.model_selection import train_test_split
from sklearn import preprocessing


df = pd.read_csv("Kickstarter.csv")

"""
The dataset can be found here:
https://webrobots.io/kickstarter-datasets/
https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2021-06-17T03_20_03_179Z.zip
"""

'\nThe dataset can be found here:\nhttps://webrobots.io/kickstarter-datasets/\nhttps://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2021-06-17T03_20_03_179Z.zip\n'

In [3]:
# df = df.drop(columns=["currency_symbol",
#                  "country_displayable_name",
#                  "created_at",
#                  "creator",
#                  "country",
#                  "urls",
#                  "currency_trailing_code",
#                  "friends",
#                  "fx_rate",
#                  "id",
#                  "is_starred",
#                  "name",
#                  "permissions"], axis = 1)

In [4]:
# df.head(3)

Do we want to combine find the length of the campaign from (created at and deadline) to see if there is value there. 

keep location, clean and ckeep city name



In [5]:
# def clean_data(text):
#     email_regex = r"\S*@\S*\s?"
#     non_alpha = "[^a-zA-Z0-9 ]"
#     multi_white_spaces = "[ ]{2,}"
#     text = re.sub(email_regex, "", text)
#     text = re.sub(non_alpha, "", text)
#     text = re.sub(multi_white_spaces, "", text)
#     return text.lower().lstrip().rstrip()

# Load The Dataset

In [6]:
# file_path = 'Downloads/Kickstarter_2021-06-17T03_20_03_179Z/Kickstarter001.csv'
file_path = "Kickstarter.csv"
# file_path_small_test = 'small_file.csv'


def clean_data(file_path):
    df = pd.read_csv(file_path)
    
    def change_to_time_series(item):
        item = datetime.datetime.fromtimestamp(item)
        return item
    
    # change the 'created-at', 'deadline', 'state_changed_at' columns
    # to time series columns
    time_series_columns = ['created_at', 'deadline', 'state_changed_at', 'launched_at']
    for column in time_series_columns:
        df[column] = df[column].apply(change_to_time_series)
    
    # dropping columns with all null values
    # for reference these the column names:
    # ['friends', 'is_backing', 'is_starred', 'permissions']
    all_null_columns = df.isnull().sum() == len(df)
#     to_drop = all_null_columns[all_null_columns == True].index
    to_drop = ['friends', 'is_backing', 'is_starred', 'permissions']
    df = df.drop(columns=to_drop, axis=1)
    
    df = df.drop(columns=["currency_symbol",
                          "country_displayable_name",
                          "creator",
                          "country",
                          "urls",
                          "source_url",
                          "currency_trailing_code",
                          "fx_rate",
                          "id",
                          "name",
                          "disable_communication",
                          "photo", # has url for photos might be useful later 
                          "usd_type", 
                          'converted_pledged_amount',
                          'pledged',
                          'usd_exchange_rate',
                          'static_usd_rate',
                          'currency', # dropping due to have pledged in USD
                          'current_currency'  # dropping due to have pledged in USD
                         ], axis=1)
    
    def extract_dictionary_info(item):
        """ Use the apply method with the column name.
        Takes in dictionary in string form,
        converts it into a dictionary, and
        returns info_type requested in a new column.
        """
        my_dic = ast.literal_eval(item)
        return my_dic

    cat_df = df['category'].apply(extract_dictionary_info).apply(pd.Series)
    cat_df['parent_name'].loc[(cat_df.parent_name.isnull() == True)& (cat_df['name'] == 'Dance')] = 'Dance'
    cat_df['parent_name'].loc[(cat_df.parent_name.isnull() == True)& (cat_df['name'] == 'Photography')] = 'Photography'
    cat_df['parent_name'].loc[(cat_df.parent_name.isnull() == True)& (cat_df['name'] == 'Games')] = 'Games'
    
    df['category_2'] = cat_df.parent_name    
    # df = pd.concat([df,pd.get_dummies(df['category_2'], dtype = float)], axis=1)
    le = preprocessing.LabelEncoder()
    df["sub_categories"] = le.fit_transform(df["category_2"])
    # 2 is Flim and Video 
    # 0 is Dance
    # 3 is Games
    # 1 is Fashion
    df.drop(columns=['category', 'category_2'], inplace=True)
      
    
    # location
    df.dropna(inplace=True)
    loc_df = df['location'].apply(json.loads).apply(pd.Series)
    df['location_2'] = loc_df.country
    df.drop(columns='location', inplace=True)
    df["sub_location"] = le.fit_transform(df["location_2"])
    # Need to find locations from the integer and add here
    df.drop(columns=['location_2'], inplace=True)

    
    # columns I can't get to work
    # in dictionary format need to work on OR text data
    # text_data need to vectorize: 'blurb
    drop_for_now = ['blurb', 'profile', 'slug']
    df.drop(columns=drop_for_now, inplace=True)
    
    
    # creating our target variable, we had the four sub-categories:
    # successful, failed, canceled, live
    # turn it into a binary variable
    df['target'] = 0
    df['target'].loc[df.state == 'successful'] = 1
    df.drop(columns='state', inplace=True)
    
    # changing boolean to integers
    df['spotlight_2'] = 0
    df['spotlight_2'].loc[df.spotlight == True] = 1

    df['staff_pick_2'] = 0
    df['staff_pick_2'].loc[df.staff_pick == True] = 1

    df['is_starrable_2'] = 0
    df['is_starrable_2'].loc[df.is_starrable == True] = 1
    df.drop(columns=['spotlight', 'staff_pick', 'is_starrable', "spotlight_2", "backers_count"],
           inplace=True)


    
    # removing the launched_at and state_changed_at time series columns
    # add at your discretion
    time_series = ['created_at', 'deadline','state_changed_at', 'launched_at', "usd_pledged"] 
    # Drop USD pledged for leakage, duh 
    df.drop(columns=time_series, inplace=True)
    
    
    return df

In [7]:
# Reading in the data and passing through the cleaning function
df = clean_data(file_path)
# df = clean_data(file_path_small_test)
df.head(3)


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in t

,goal,sub_categories,sub_location,target,staff_pick_2,is_starrable_2
0,15000.0,3,2,0,0,1
1,10000.0,2,61,1,0,0
2,3000.0,0,61,1,0,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3660 entries, 0 to 3661
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   goal            3660 non-null   float64
 1   sub_categories  3660 non-null   int64  
 2   sub_location    3660 non-null   int64  
 3   target          3660 non-null   int64  
 4   staff_pick_2    3660 non-null   int64  
 5   is_starrable_2  3660 non-null   int64  
dtypes: float64(1), int64(5)
memory usage: 200.2 KB


# Split the Data

In [9]:
df.corr()

,goal,sub_categories,sub_location,target,staff_pick_2,is_starrable_2
goal,1.000000,0.022875,-0.015123,-0.056283,-0.010161,-0.005657
sub_categories,0.022875,1.000000,-0.153201,-0.102882,-0.161327,0.290464
sub_location,-0.015123,-0.153201,1.000000,0.136986,0.093118,-0.078789
target,-0.056283,-0.102882,0.136986,1.000000,0.215084,-0.356840
staff_pick_2,-0.010161,-0.161327,0.093118,0.215084,1.000000,-0.058756
is_starrable_2,-0.005657,0.290464,-0.078789,-0.356840,-0.058756,1.000000


In [10]:
# Split the data to avoid leakage
x = df.drop(columns=['target'])
y = df['target']

In [11]:
# Create a training and validation set
# Smaller test size is acceptable due to the amount of data available
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=.1, random_state=42)

print('x_train:', len(X_train))
print('x_val:', len(X_val))
print('y_train:',y_train.shape)
print('y_val:', y_val.shape)

x_train: 3294
x_val: 366
y_train: (3294,)
y_val: (366,)


# Set Model Baseline

In [12]:
baseline = y_train.value_counts(normalize = True).max()
print("The baseline score for our dataset is:", baseline)

The baseline score for our dataset is: 0.6900425015179114


# Explore Models

In [13]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras.callbacks import TensorBoard

from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# required for compatibility bewteen sklearn and keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

## Configure a Neural Network ##

In [14]:
# Define input dimensions to avoid hidden layer issues
input_dims = X_train.shape[1]
input_dims

5

## Create Model Function ##
This function should help you save time on creating more models.

In [15]:
# Function to create model, required for KerasClassifier
def create_model(units = 128, optimizer = "adam", activation = "sigmoid"):
    """"
    Returns a complied keras model 
    
    Parameters
    ----------
    units: int 
        number of neruons/nodes/units to use in each hidden layer
        
    Returns
    -------
    model: keras object 
    """

    model = Sequential()
    model.add(Dense(units = 64, input_dim = input_dims, activation = activation))
    model.add(Dense(units = 32, activation = activation))
    model.add(Dense(units = 10, activation = activation))   
    model.add(Dense(1, activation="sigmoid")) # USE SIGMOID FOR BINARY CLASSIFICATION
    model.compile(loss="binary_crossentropy",
                    optimizer = "adam",
                    metrics=["accuracy"])
    return model

In [16]:
# Instantiate a base model for grid search
base_model = KerasClassifier(build_fn = create_model)

### Perform a Grid Search To Optimize Our Model 

In [17]:
# define the grid search parameters
param_grid = {'batch_size': [32],
              'epochs': [3],
              'units':[32, 64]}

In [18]:
# Create Grid Search
grid = GridSearchCV(estimator = base_model,
                    param_grid = param_grid, 
                    n_jobs=-2, 
                    verbose=1, 
                    cv=3)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Epoch 1/3
103/103 [==============================] - 1s 2ms/step - loss: 0.6776 - accuracy: 0.5835
Epoch 2/3
103/103 [==============================] - 0s 2ms/step - loss: 0.6207 - accuracy: 0.6900
Epoch 3/3
103/103 [==============================] - 0s 2ms/step - loss: 0.6193 - accuracy: 0.6900
Best: 0.6900424957275391 using {'batch_size': 32, 'epochs': 3, 'units': 32}
Means: 0.6900424957275391, Stdev: 0.010312853587184763 with: {'batch_size': 32, 'epochs': 3, 'units': 32}
Means: 0.6900424957275391, Stdev: 0.010312853587184763 with: {'batch_size': 32, 'epochs': 3, 'units': 64}


In [19]:
# Assign the best model 
best_nn_model = grid_result.best_estimator_
# Confirm the models params
best_nn_model.get_params()

{'batch_size': 32,
 'epochs': 3,
 'units': 32,
 'build_fn': <function __main__.create_model(units=128, optimizer='adam', activation='sigmoid')>}

In [20]:
    best_nn_model.fit(X_train, 
                        y_train, 
                        validation_data = (X_val,y_val),
                        verbose=2)

Epoch 1/3
103/103 - 1s - loss: 0.6205 - accuracy: 0.6900 - val_loss: 0.6000 - val_accuracy: 0.7131
Epoch 2/3
103/103 - 0s - loss: 0.6199 - accuracy: 0.6900 - val_loss: 0.6008 - val_accuracy: 0.7131
Epoch 3/3
103/103 - 0s - loss: 0.6198 - accuracy: 0.6900 - val_loss: 0.6026 - val_accuracy: 0.7131


### Check the models scores

In [21]:
best_nn_model_val_score = best_nn_model.score(X_val,y_val)
print("The best NN model validation score is:", best_nn_model_val_score)

12/12 [==============================] - 0s 1ms/step - loss: 0.6026 - accuracy: 0.7131
The best NN model validation score is: 0.7131147384643555


# Create A Classifier Model 
Compare this model with our NN

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

In [23]:
clf_model = make_pipeline(
                        RandomForestClassifier(
                                                random_state= 42,
                                                ))

clf_model.fit(X_train, y_train)

Pipeline(steps=[('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])

### Check the CLF models score

In [24]:
clf_model_val_score = clf_model.score(X_val,y_val)
print("The best CLF model validation score is:", clf_model_val_score)

The best CLF model validation score is: 0.7896174863387978


In [25]:
dir(clf_model)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_fit_params',
 '_check_n_features',
 '_estimator_type',
 '_final_estimator',
 '_fit',
 '_get_param_names',
 '_get_params',
 '_get_tags',
 '_inverse_transform',
 '_iter',
 '_log_message',
 '_more_tags',
 '_pairwise',
 '_replace_estimator',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_set_params',
 '_sk_visual_block_',
 '_transform',
 '_validate_data',
 '_validate_names',
 '_validate_steps',
 'classes_',
 'decision_function',
 'fit',
 'fit_predict',
 'fit_tr

In [26]:
import joblib
joblib.dump(clf_model, "my_h5_model.h5")

['my_h5_model.h5']

In [27]:
# # Save the entire model as a SavedModel.
# !mkdir -p saved_model
# clf_model.save('my_h5_model.h5') 